In [1]:
# Libraries
# =================
import os
import pandas as pd
import numpy as np

# Parameters
# =================
from src.parameters import Parameters

# Data
# =================
from src.data_clean import get_data
from src.data_ranking import atp_ranking_last_date
from src.data_scrapped_clean import scrapped_data_organized

# Scrapping
# =================
from src.scrapping import scrapping_tennis_data


# Classification
# =================
from src.class_preprocessing import data_to_class
from src.class_classification import train_classification, fn_classification

# Risk
# =================
from src.risk_analysis import risk_analysis_montecarlo

# Optimization
# =================
from src.optimization import Optimization

# Warnings
# =================
import warnings
warnings.filterwarnings("ignore")

In [2]:
# # Date to process
# # =================
# year, month, day= '2022', '03', '16'
# n_date = f'{year}-{month}-{day}'
# ndate = f'{year}{month}{day}'

In [3]:
# # Train
# # =================
# df_raw = pd.read_csv(os.path.join(Parameters.train_path, Parameters.nombre_archivo_raw))
# df_raw = df_raw.rename(columns={'AvgW':'pl1_bet','AvgL':'pl2_bet'})
# df_bronze, df_silver, df_gold = get_data(df_raw, 
#                                         True, 
#                                         Parameters.train_path, 
#                                         Parameters.nombre_archivo_gold)

# X, y, preprocessor = data_to_class(df_gold)
# train_classification(X, y, preprocessor, Parameters.results_path, Parameters.models_path)

In [4]:
# # Scrapping
# # =================
# df_games_of_day, df_games_acum = scrapping_tennis_data(year,month,day,
#                                                     Parameters.req_headers,
#                                                     Parameters.scraped_path, 
#                                                     Parameters.file_match_result, 
#                                                     Parameters.file_fields_desc,
#                                                     Parameters.file_players_desc,
#                                                     Parameters.file_games, 
#                                                     Parameters.daily_dump_path)

In [5]:
# df_games_of_day = pd.read_csv(os.path.join(Parameters.daily_dump_path, 'df_games_20220306.csv'), sep='|')

In [6]:
# # Pred
# # =================

# atp_all = atp_ranking_last_date(Parameters.utils_path, "atp_mens_tour", n_date)
# df_fply, df_fply_bronze, df_fply_silver, df_fply_gold = scrapped_data_organized(df_games_of_day,
#                                                                                 Parameters.utils_path, 
#                                                                                 Parameters.daily_dump_path,
#                                                                                 Parameters.file_paises,
#                                                                                 f'df_games_cleaned_{ndate.replace("-","")}.csv',
#                                                                                 atp_all)

# # df_to_pred                              = pd.read_csv(os.path.join(results_path, file_data_scrapped_cleaned))
# df_to_pred                              = df_fply_gold.copy()
# X, y, preprocessor                      = data_to_class(df_to_pred)
# df_class, \
# df_class_consolidate, \
# reporte_clasificacion                   = fn_classification(df_to_pred, X, n_date, Parameters.results_path)

In [7]:
# # Risk
# # =================
# df_risk, monte_carlo_results, monte_carlo_summary = risk_analysis_montecarlo(df_class,
#                                                                             Parameters.risk_free_rate, 
#                                                                             Parameters.risk_tolerance, 
#                                                                             Parameters.total_money, 
#                                                                             Parameters.num_simulations, 
#                                                                             Parameters.num_bets,
#                                                                             Parameters.plots_path,
#                                                                             Parameters.results_path,
#                                                                             f'{Parameters.name_monte_carlo_dist}_{ndate}.png',
#                                                                             f'{Parameters.name_ev_comparation}_{ndate}.png',
#                                                                             f'{Parameters.name_sharpe_ratio_comparison}_{ndate}.png',
#                                                                             f'{Parameters.file_betting_analysis}_{ndate}.xlsx',
#                                                                             False)

In [8]:
# # Optimization
# # =================

# # df_risk = pd.read_excel(os.path.join(Parameters.results_path,
# #                                               f'{Parameters.file_betting_analysis}_{ndate}.xlsx'),
# #                                               sheet_name='Betting Decisions')

# # df_risk['match'] = df_risk['pl1'] +'\n' + df_risk['pl2']

# optimizer = Optimization(df_risk, Parameters.total_money, Parameters.max_loss_percentage, Parameters.min_percentage)
# df_risk_optimized = optimizer.optimize()
# df_risk_optimized = risk_analysis_montecarlo(df_risk_optimized,
#                                             Parameters.risk_free_rate, 
#                                             Parameters.risk_tolerance, 
#                                             Parameters.total_money, 
#                                             Parameters.num_simulations, 
#                                             Parameters.num_bets,
#                                             Parameters.plots_path,
#                                             Parameters.results_path,
#                                             f'{Parameters.name_monte_carlo_dist}_{ndate}__Optimized.png',
#                                             f'{Parameters.name_ev_comparation}_{ndate}_Optimized.png',
#                                             f'{Parameters.name_sharpe_ratio_comparison}_{ndate}_Optimized.png',
#                                             f'{Parameters.file_betting_analysis}_{ndate}_Optimized.xlsx',
#                                             True)

In [9]:
# start_date = pd.to_datetime('2022-03-01')
# end_date = pd.to_datetime('2022-03-17')
# list_dates = pd.date_range(start=start_date, end=end_date).tolist()
# total_money_daily = Parameters.total_money / len(list_dates)

# df_class = pd.read_excel(os.path.join(Parameters.results_path,'df_class.xlsx'))

# for f, date in enumerate(list_dates):

#     year, month, day= str(list_dates[f].year), \
#                     str(list_dates[f].month).zfill(2), \
#                     str(list_dates[f].day).zfill(2)

#     n_date = f'{year}-{month}-{day}'
#     ndate = f'{year}{month}{day}'
#     df_class_date = df_class[(df_class['date_run'] == date.strftime('%Y-%m-%d'))]
#     # Risk
#     # =================
#     df_risk = risk_analysis_montecarlo(df_class_date,
#                                         Parameters.risk_free_rate, 
#                                         Parameters.risk_tolerance, 
#                                         total_money_daily, 
#                                         Parameters.num_simulations, 
#                                         Parameters.num_bets,
#                                         Parameters.plots_path,
#                                         Parameters.results_path,
#                                         f'{Parameters.name_monte_carlo_dist}_{ndate}.png',
#                                         f'{Parameters.name_ev_comparation}_{ndate}.png',
#                                         f'{Parameters.name_sharpe_ratio_comparison}_{ndate}.png',
#                                         f'{Parameters.file_betting_analysis}_{ndate}.xlsx',
#                                         False)
        
        
#     optimizer = Optimization(df_risk, total_money_daily, Parameters.max_loss_percentage, Parameters.min_percentage)
#     df_risk_optimized = optimizer.optimize()
#     df_risk_optimized = risk_analysis_montecarlo(df_risk_optimized,
#                                                 Parameters.risk_free_rate, 
#                                                 Parameters.risk_tolerance, 
#                                                 total_money_daily, 
#                                                 Parameters.num_simulations, 
#                                                 Parameters.num_bets,
#                                                 Parameters.plots_path,
#                                                 Parameters.results_path,
#                                                 f'{Parameters.name_monte_carlo_dist}_{ndate}__Optimized.png',
#                                                 f'{Parameters.name_ev_comparation}_{ndate}_Optimized.png',
#                                                 f'{Parameters.name_sharpe_ratio_comparison}_{ndate}_Optimized.png',
#                                                 f'{Parameters.file_betting_analysis}_{ndate}_Optimized.xlsx',
#                                                 True)

In [10]:
start_date = pd.to_datetime('2024-09-01')
end_date = pd.to_datetime('2024-09-10')
list_dates = pd.date_range(start=start_date, end=end_date).tolist()

df_class = pd.read_excel(os.path.join(Parameters.results_path,'df_class.xlsx'))

df_class_all = pd.DataFrame()
for f, date in enumerate(list_dates):

    year, month, day= str(list_dates[f].year), \
                    str(list_dates[f].month).zfill(2), \
                    str(list_dates[f].day).zfill(2)
    n_date = f'{year}-{month}-{day}'
    ndate = f'{year}{month}{day}'
                    
    syear, smonth, sday= str(start_date.year), \
                    str(start_date.month).zfill(2), \
                    str(start_date.day).zfill(2)
    s_date = f'{syear}-{smonth}-{sday}'
    sdate =  f'{syear}{smonth}{sday}'
                    
    eyear, emonth, eday= str(end_date.year), \
                    str(end_date.month).zfill(2), \
                    str(end_date.day).zfill(2)
    e_date =  f'{eyear}-{emonth}-{eday}'
    edate = f'{eyear}{emonth}{eday}'
    
    df_class_date = df_class[(df_class['date_run'] == date.strftime('%Y-%m-%d'))]
    df_class_date['date'] = ndate
    
    df_class_all = pd.concat([df_class_all, df_class_date], axis = 0 )

# Risk
# =================
df_risk = risk_analysis_montecarlo(df_class_all,
                                    Parameters.risk_free_rate, 
                                    Parameters.risk_tolerance, 
                                    Parameters.total_money, 
                                    Parameters.num_simulations, 
                                    Parameters.num_bets,
                                    Parameters.plots_path,
                                    Parameters.results_path,
                                    f'{Parameters.name_monte_carlo_dist}_{sdate}_{edate}.png',
                                    f'{Parameters.name_ev_comparation}_{sdate}_{edate}.png',
                                    f'{Parameters.name_sharpe_ratio_comparison}_{sdate}_{edate}.png',
                                    f'{Parameters.file_betting_analysis}_{sdate}_{edate}.xlsx',
                                    False)

optimizer = Optimization(df_risk, Parameters.total_money, Parameters.max_loss_percentage, Parameters.min_percentage)
df_risk_optimized = optimizer.optimize()
df_risk_optimized = risk_analysis_montecarlo(df_risk_optimized,
                                            Parameters.risk_free_rate, 
                                            Parameters.risk_tolerance, 
                                            Parameters.total_money, 
                                            Parameters.num_simulations, 
                                            Parameters.num_bets,
                                            Parameters.plots_path,
                                            Parameters.results_path,
                                            f'{Parameters.name_monte_carlo_dist}_{sdate}_{edate}__Optimized.png',
                                            f'{Parameters.name_ev_comparation}_{sdate}_{edate}_Optimized.png',
                                            f'{Parameters.name_sharpe_ratio_comparison}_{sdate}_{edate}_Optimized.png',
                                            f'{Parameters.file_betting_analysis}_{sdate}_{edate}_Optimized.xlsx',
                                            True)

Análisis completado. Revisa el archivo betting_analysis_20240901_20240910.xlsx' para ver los resultados.
Análisis completado. Revisa el archivo betting_analysis_20240901_20240910_Optimized.xlsx' para ver los resultados.


In [11]:
df_all_optimized = df_risk_optimized.copy()
num_apuestas = len(df_all_optimized)
df_all_optimized['Money_earned'] = df_all_optimized['Money_to_Bet'] * df_all_optimized['Payout']
df_winner = df_all_optimized[(df_all_optimized['Result'] == df_all_optimized['Class'])]
df_winner.reset_index(drop=True,  inplace=True)
alpha = df_all_optimized.groupby(['date'])[['Money_earned']].sum().rename(columns={'Money_earned':'Possible_Money_earned'})
betha = df_all_optimized.groupby(['date'])[['match']].count().rename(columns={'match':'Num_bets'})
gamma = df_winner.groupby(['date'])[['Money_earned']].sum()
delta = df_winner.groupby(['date'])[['match']].count().rename(columns={'match':'Num_winner_bets'})

iota = betha.join(delta).fillna(0)
iota['Num_winner_bets'] = iota['Num_winner_bets'].astype(int)
iota['perc_winner_bets'] = np.where(iota['Num_winner_bets'] == 0, 0,  iota['Num_winner_bets'] / iota['Num_bets'])
iota['perc_winner_bets'] = round(iota['perc_winner_bets']*100,2).astype(str) + '%'

iota = iota.join(alpha).join(gamma).fillna(0)
iota['perc_Money_earned'] = np.where(iota['Money_earned'] == 0, 0,  iota['Money_earned'] / iota['Possible_Money_earned'])
iota['perc_Money_earned'] = (iota['perc_Money_earned']*100).astype(int).astype(str) + '%'

Money_earned = iota['Money_earned'].sum()
num_apuestas_victoriosas = iota['Num_winner_bets'].sum()
dinero_ganado_perdido = Parameters.total_money - Money_earned
if dinero_ganado_perdido < 0:
    str_dinero_ganado_perdido = f'ganaste {dinero_ganado_perdido*-1}'
    str_patrimonio = f'Aumentando tu patrimonio un {(Parameters.total_money - Money_earned)/Parameters.total_money*-1}'
    
else:
    str_dinero_ganado_perdido = f'perdiste {dinero_ganado_perdido}'
    str_patrimonio = f'Destruyendo tu patrimonio un {(Parameters.total_money - Money_earned)/Parameters.total_money}'
    
print(f"Durante el {start_date.strftime('%Y-%m-%d')} y el {end_date.strftime('%Y-%m-%d')}, hiciste {num_apuestas} apuestas\n\
de las cuales ganaste en {num_apuestas_victoriosas},\n\
es decir, ganaste un {round((num_apuestas_victoriosas/num_apuestas)*100,1)}% de las veces \n\
Obtuviste una ganancia {int(Money_earned)}\n\
Tu inversión eran {Parameters.total_money}\n\
Por ende, {str_dinero_ganado_perdido}\n\
{str_patrimonio}\n\
Tu patrimonio quedó en {Money_earned}\n\
    ")
iota

Durante el 2024-09-01 y el 2024-09-10, hiciste 69 apuestas
de las cuales ganaste en 65,
es decir, ganaste un 94.2% de las veces 
Obtuviste una ganancia 99
Tu inversión eran 100
Por ende, perdiste 0.24200000000001864
Destruyendo tu patrimonio un 0.0024200000000001863
Tu patrimonio quedó en 99.75799999999998
    


,Num_bets,Num_winner_bets,perc_winner_bets,Possible_Money_earned,Money_earned,perc_Money_earned
date,,,,,,
20240903,14,14,100.0%,20.7660,20.7660,100%
20240904,18,18,100.0%,27.9750,27.9750,100%
20240905,8,7,87.5%,12.1698,9.8148,80%
20240906,2,2,100.0%,3.1200,3.1200,100%
20240907,1,0,0.0%,1.5900,0.0000,0%
20240909,14,12,85.71%,27.0150,19.2750,71%
20240910,12,12,100.0%,18.8072,18.8072,100%
